# DM_05_04

## Importar bibliotecas

In [ ]:
%matplotlib inline
import pylab
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope

pylab.rcParams.update({'font.size': 14})

## Ler CSV

In [ ]:
df = pd.read_csv("AnomalyData.csv")
df.head()

Salvar state_code para rotular valores discrepantes. "data" contém apenas variáveis quantitativas.

In [ ]:
state_code = df["state_code"]
data = df.loc[:, "data science": "Openness"]

## Valores discrepantes univariados 
Criar diagrama em caixa para exibir valores discrepantes univariados sobre "modern dance."

In [ ]:
param = "modern dance"

Obter valores dos quantis e amplitude interquartis dos limites dos valores discrepantes.

In [ ]:
qv1 = data[param].quantile(0.25)
qv2 = data[param].quantile(0.5)
qv3 = data[param].quantile(0.75)
qv_limit = 1.5 * (qv3 - qv1)

Obter posição dos valores discrepantes e usar state_code para rótulos.

In [ ]:
un_outliers_mask = (data[param] > qv3 + qv_limit) | (data[param] < qv1 - qv_limit)
un_outliers_data = data[param][un_outliers_mask]
un_outliers_name = state_code[un_outliers_mask]

Criar diagrama em caixa usando "modern dance."

In [ ]:
fig = pylab.figure(figsize=(4,6))
ax = fig.add_subplot(1, 1, 1)
for name, y in zip(un_outliers_name, un_outliers_data):
    ax.text(1, y, name)
ax.boxplot(data[param])
ax.set_ylabel(param)

##  Valores discrepantes bivariados
Criar diagrama de dispersão com elipse como limite para valores discrepantes.

Usar os termos de pesquisa no Google "data science" e "ceo" neste exemplo.

In [ ]:
params = ["data science", "ceo"]
params_data = np.array([df[params[0]], df[params[1]]]).T

Computar o "envelope elíptico."

In [ ]:
ee = EllipticEnvelope()
ee.fit(params_data)

Obter o nome e a posição dos valores discrepantes.

In [ ]:
biv_outliers_mask = ee.predict(params_data) == -1
biv_outliers_data = params_data[biv_outliers_mask]
biv_outliers_name = state_code[biv_outliers_mask]

Calcular o limite de decisão do diagrama de dispersão.

In [ ]:
xx, yy = np.meshgrid(np.linspace(params_data[:, 0].min(), params_data[:, 0].max(), 100),
                     np.linspace(params_data[:, 1].min(), params_data[:, 1].max(), 100))
zz = ee.decision_function(np.c_[xx.ravel(), yy.ravel()])
zz = zz.reshape(xx.shape)

Traçar diagrama de dispersão com envelope elíptico e rotular valores discrepantes.

In [ ]:
fig = pylab.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
for name, xy in zip(biv_outliers_name, biv_outliers_data):
    ax.text(xy[0], xy[1], name)
ax.contour(xx, yy, zz, levels=[0], linewidths=2)
ax.scatter(params_data[:, 0], params_data[:, 1], color='black')
ax.set_xlabel(params[0])
ax.set_ylabel(params[1])

##  Valores discrepantes multivariados
Usar o algoritmo para máquinas de vetores de suporte (SVM) de uma classe para classificar casos incomuns.

In [ ]:
ocsvm = OneClassSVM(nu=0.25, gamma=0.05)
ocsvm.fit(data)

Listar nomes dos estados com valores discrepantes com base na SVM de uma classe.

In [ ]:
# 
state_code[ocsvm.predict(data) == -1]